# Dataset Deduplication Tutorial


## Problem Statement

One of the first steps in Exploratory Data Analysis (EDA) is to check for duplicates. Duplicates add no new information and can distort model training by over-emphasizing features that in appear in the duplicates.

DataEval provides a Duplicates class to assist you in removing duplicates so you can start training your models on high quality data.


### _When to use_

The Duplicates class should be used if you need to find duplicate images in your dataset.


### _What you will need_

1. A python envornment with following packages installed:
   - dataeval or dataeval[all]
2. A dataset to analyze


## _Getting Started_

Let's import the required libraries needed to set up a minimal working example


In [ ]:
# Google Colab Only
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval
except Exception:
    pass

In [ ]:
import numpy as np
from torch.utils.data import Subset

from dataeval.detectors.linters import Duplicates
from dataeval.utils.data import Metadata
from dataeval.utils.data.datasets import MNIST

## Loading in the data

Load the MNIST data and create the dataset.

The MNIST dataset contains 70,000 images - 60,000 in the train set and 10,000 in the test set.
For the purposes of this demonstration, we are just going to use the test set.


In [ ]:
# Load in the mnist dataset
testing_dataset = MNIST(root="./data/", image_set="test")

# Get the labels
labels = Metadata(testing_dataset).targets.labels

Because the MNIST dataset does not contain any exact duplicates we are going to adjust the dataset to include some.


In [ ]:
# Creating some indices to duplicate
print("Exact duplicates")
duplicates = {}
for i in [1, 2, 5, 9]:
    matching_indices = np.where(labels == i)[0]
    print(f"\t{i} - ({matching_indices[23]}, {matching_indices[78]})")
    duplicates[int(matching_indices[78])] = int(matching_indices[23])

In [ ]:
# Create a subset with the identified duplicate indices swapped
indices_with_duplicates = [duplicates.get(i, i) for i in range(len(testing_dataset))]
duplicates_ds = Subset(testing_dataset, indices_with_duplicates)

## Finding the Duplicates

Now we are asking our Duplicates class to find the needle in the haystack.
There are only 4 exact duplicates.


In [ ]:
# Initialize the Duplicates class to begin to identify duplicate images.
identifyDuplicates = Duplicates()

# Evaluate the data
results = identifyDuplicates.evaluate(duplicates_ds)

The results can be returned as a dictionary with exact and near as the keys. So we will extract those to view the results.


In [ ]:
for category, images in results.dict().items():
    print(f"{category} - {len(images)}")
    print(f"\t{images}")

The `Duplicates` class was able to find all 4 exact duplicates out of the 10,000 samples.

It also found several sets of images that are very closely related to each other, and since we are using hand written digits we would expect it to find some images that were nearly identical.


In [ ]:
### TEST ASSERTION CELL ###
assert len(results.exact) == len(duplicates)
for k, v in duplicates.items():
    assert [v, k] in results.exact